# thorskyclasses -- a convenient overlay for astropy.coordinates

## John Thorstensen

**thorskyclasses3** is a set of convenience functions that sit atop ```astropy.coordinates, astropy.time```, and ```astropy.units```.  In addition, it calls functions from ```thorskyutils```, which include functionality not included in astropy, and also some routines for relatively lightweight, accurate-enough computations of such things as the local sidereal time and the positions of the moon and sun. 

The module also imports from the python ```dateutils``` and the time zone library ```pytz```.  

The main aim is to provide the infrastructure necessary for a new, all-python version of skycalc, which will have
a GUI interface similar to JSkyCalc.  However, the same infrastructure is very useful for such tasks as figuring out which of your data were taken with the moon up, or in twilight, or whether the barycentric corrections are right, and many other applications.  

Two data files are needed, ```cartesian_bright.dat``` and ```observatories_rev.dat```.  These list respectively the brightest stars and data on the canned observing sites; in both cases the coordinates used are cartesian.

###  The Observation class

The main concept in ```thorskyclasses3``` is the ```Observation``` subclass.  As its name implies, this is meant to model a single observation, taken from a site on earth, looking toward specified celestial coordinates, and taken at a specified time.  Accordingly an ```Observation``` has these three main attributes:
1. ```site```, which is a site on earth comprising of an astropy EarthLocation, a name, and some other data such as time zone;
2. ```celest``` which is an astropy SkyCoord object; and 
3.  ```t```, which is an astropy Time object.

When an ```Observation``` is initialized, these inputs are specified as keyword arguments, any of which can be omitted.  Instantiating an ```Observation``` with no arguments 
1. sets the ```site``` to a default observatory (MDM in my case), 
2. sets the time ```t``` to the present computer clock time, and  
3. sets the sky position ```celest``` to correspond to the zenith at that site and time.

Here is an example.  Note that native ```astropy``` functions are used for formatted output. 
```thorskyclasses3``` provides little in the way of output formatting, since astropy already handles this.

NOTE that the "ErfaWarning" will appear rather frequently, because ERFA routines called by astropy depend on up-to-date information on such things as small fluctuations in the earth orientation parameters.  These effects are too small to matter for nearly all users, so the warning can be ignored unless you require extremely precise absolute positions.
       
Here is an example of instantiating an ```Observation``` without arguments.  Your output will vary depending on when you run it!

In [1]:
from thorskyclasses3 import *
# thorskyclasses imports astropy.units as u

obs = Observation()  # instantiates to right now, straight up at default observatory.

# Let's see what we got:
print("RA: ",obs.celest.ra.to_string(unit = u.hourangle, sep = ' ',precision=2), ",  dec: ",
      obs.celest.dec.to_string(sep = ' ', precision = 1, pad = True, alwayssign=True))
print(obs.celest.frame.name, obs.celest.frame.equinox)
print(obs.site.name,obs.site.location.lon.to_string(),obs.site.location.lat.to_string())
dt = obs.t.to_datetime()
print(dt.strftime("%a  %Y-%m-%d  %H:%M:%S"), " UT")


RA:  17 53 59.89 ,  dec:  +31 57 00.0
fk5 J2018.670
Kitt Peak [MDM Obs] -111d36m59.9916s 31d56m59.9839s
Tue  2018-09-04  02:27:47  UT


The ```observation``` class provides several methods for changing the input parameters.  

To change sites, use the ```setsite``` method and the short observatory code you'll find in the observatory list.

In [2]:
obs.setsite('aao')  
print(obs.site.name,obs.site.location.lon.to_string(),obs.site.location.lat.to_string())

Anglo-Australian Obs 149d03m57.9322s -31d16m37.3359s


The ```setcelest``` method lets you change the celestial position; there are many valid formats, for example:

In [31]:
obs.setcelest("18:22:22.23  -14:15:17.9  1950.")
print("RA: ",obs.celest.ra.to_string(unit = u.hourangle, sep = ' ',precision=2), ",  dec: ",
      obs.celest.dec.to_string(sep = ' ', precision = 1, pad = True, alwayssign=True))
print(obs.celest.frame.name, obs.celest.frame.equinox)
print(" ")
obs.setcelest((14.234, 15.25))
print("RA: ",obs.celest.ra.to_string(unit = u.hourangle, sep = ' ',precision=2), ",  dec: ",
      obs.celest.dec.to_string(sep = ' ', precision = 1, pad = True, alwayssign=True))
print(obs.celest.frame.name)
print(" ")
obs.setcelest(('17.325d','28:33:52'))
print("RA: ",obs.celest.ra.to_string(unit = u.hourangle, sep = ' ',precision=2), ",  dec: ",
      obs.celest.dec.to_string(sep = ' ', precision = 1, pad = True, alwayssign=True))
print(obs.celest.frame.name)



RA:  18 22 22.23 ,  dec:  -14 15 17.9
fk5 J1950.000
 
RA:  14 14 02.40 ,  dec:  +15 15 00.0
icrs
 
RA:  1 09 18.00 ,  dec:  +28 33 52.0
icrs


Notice that coordinates without an equinox default to the ICRS system, which is essentially identical
to J2000 but does not formally include an equinox.

To change the time of observation, use the ```settime``` method, which again accepts a range of formats. The default is to input times in local zone time, but internally it is alway stored as universal time. Months unfortunately need to be given by numbers rather than names or abbreviations.  You can input universal time by setting the keyword argument ```use_local_time``` to ```False```.

In [41]:
obs.settime("2018-09-14 13:14:15")   # local time
print(obs.t.to_datetime().strftime("%a  %Y-%m-%d  %H:%M:%S"))  # internally, stored as UT
print(" ")
obs.settime((2018,9,14,13,14,15))
print(obs.t.to_datetime().strftime("%a  %Y-%m-%d  %H:%M:%S"))
print(" ")
obs.settime("2018-09-14 13:14:15", use_local_time = False)  # input a UT
print(obs.t.to_datetime().strftime("%a  %Y-%m-%d  %H:%M:%S"))  # and it's stored as-is.

Fri  2018-09-14  03:14:15
 
Tue  2018-09-04  03:14:15
 
Fri  2018-09-14  13:14:15


The conversion from ISO dates ("2019-04-17 22:18:13") to the internal format depends on python's ```dateutils```.

## How about some results? 

Several methods compute results, which are stored in class attributes. The first, ```computesky```, computes the hour angle, airmass, altitude and azimuth, and parallactic angle.  Where appropriate, these are stored as astropy Angles, which are unit-aware. To illustrate I'm printing the raw representations:

In [44]:
obs.computesky()
print("hour angle is an Angle: ",obs.hanow)
print("airmass is pure number:",obs.airmass)
print("altitude and azimuth are angles:",obs.altit, obs.az)
print("parallactic angle:",obs.parang)

hour angle is an Angle:  -2h25m56.2626s
airmass is pure number: 2.802415820000603
altitude and azimuth are angles: 0.361762rad 0.591783rad
parallactic angle: -2.56716rad


After running ```computesky```, you can compute quantities related to the sun and moon, if you need them, with ```computesunmoon```. This also causes, where appropriate, computation of moon-object angle, the moon-sun angle, the lunar contribution to the estimated nighttime sky brightness, and during twilight an estimate of the sky brightness enhancement due to twilight.  The values computed are astropy quantities (e.g., ```SkyCoord```s and ```Angle```s) when appropriate.  A verbal description of the moon phase is generated, too.  Here is just a sample of what's available:

In [46]:
obs.computesunmoon()
print("sun:",obs.sunpos)
print("sun alt az:",obs.sunaltit,obs.sunaz)
print("twilight in magnitudes:",obs.twi)
print("moon:",obs.moonpos)
print("moon alt az:",obs.moonaltit,obs.moonaz)
print("obs.moonphasedescr:",obs.moonphasedescr)
print("obs.lunsky (lunar sky brightness in mag/sq arcsec):",obs.lunsky)
print("obs.moonillumfrac, obs.moonobjang:",obs.moonillumfrac,obs.moonobjang)

sun: <SkyCoord (PrecessedGeocentric: equinox=J2018.700, obstime=J2000.000, obsgeoloc=( 0.,  0.,  0.) m, obsgeovel=( 0.,  0.,  0.) m / s): (ra, dec) in deg
    ( 172.35698726,  3.29970231)>
sun alt az: -1.04857rad 3.54309rad
twilight in magnitudes: 0.0
moon: <SkyCoord (PrecessedGeocentric: equinox=J2018.700, obstime=J2000.000, obsgeoloc=( 0.,  0.,  0.) m, obsgeovel=( 0.,  0.,  0.) m / s): (ra, dec) in deg
    ( 231.92462228, -13.4770257)>
moon alt az: -0.15255rad 4.33415rad
obs.moonphasedescr: 2.4 days before first quarter
obs.lunsky (lunar sky brightness in mag/sq arcsec): 99.0
obs.moonillumfrac, obs.moonobjang: 0.2608289864385164 144d42m57.1113s


The events of a single night -- sunset, sunrise, twilights, and so on -- are computed with the ```setnightevents``` method.  The times are astropy ```Time```s.  To be easily intelligible as local times they can be converted to python datetimes; I've also provided a function ```time_rounded_to_minute``` that rounds off these rather ragged events (refraction variations near the horizon make rise-set times imprecise).  The night for which the events are computed changes over around local noon, so if the time given by ```t``` is in the wee hours of the morning, the events are computd for the correct night.  

In [50]:
obs.setnightevents()
print("obs.tsunset:",obs.tsunset)
print("obs.tevetwi:",obs.tevetwi)

tz = obs.site.localtz   # for brevity

print(" ")        
# in more legible form ...
sunset = obs.tsunset.to_datetime(timezone = tz)
print("         Sunset:  %s" % (time_rounded_to_minute(sunset, incl_date = True, incl_day = True)))
endtwi = obs.tevetwi.to_datetime(timezone = tz) 
print("  Twilight Ends:  %s" % (time_rounded_to_minute(endtwi, incl_date = True, incl_day = True)))
nghtctr = obs.tnightcenter.to_datetime(timezone = tz)
print("Center of Night:  %s" % (time_rounded_to_minute(nghtctr, incl_date = True, incl_day = True)))
begtwi = obs.tmorntwi.to_datetime(timezone = tz)
print("Twilight Begins:  %s" % (time_rounded_to_minute(begtwi, incl_date = True, incl_day = True)))
sunrise = obs.tsunrise.to_datetime(timezone = tz)
print("        Sunrise:  %s" % (time_rounded_to_minute(sunrise, incl_date = True, incl_day = True)))


obs.tsunset: 2018-09-14 07:55:02.342220
obs.tevetwi: 2018-09-14 09:15:34.272438
 
         Sunset:  Fri 2018-09-14 17:55
  Twilight Ends:  Fri 2018-09-14 19:16
Center of Night:  Fri 2018-09-14 23:59
Twilight Begins:  Sat 2018-09-15 04:43
        Sunrise:  Sat 2018-09-15 06:03


Planetary information can be updated with ```computeplanets```.  This generates a dictionary of the planetary positions *in the present equinox*; the keys are the lowercase names of the planets.  It also generates estimates of the V magnitudes, taking into account the empirical phase functions for the planets through Mars.

In [53]:
obs.computeplanets()
print(obs.planetdict['mars'].to_string('hmsdms'))
print(obs.planetmags['mars'])

20h16m25.1121s -24d44m12.0447s
-1.74833088571


Finally, barycentric corrections are computed with ```computebary```, which also creates an attribute ```tbary``` which is the same as ```t``` but with the time correction added.  Note the unit conversions forced here with the quantity's ```to``` method.

In [7]:
obs.computebary()
print("obs.barytcorr in seconds: ",obs.barytcorr.to(u.s))
print("obs.baryvcorr in km/s:",obs.baryvcorr.to(u.km/u.s))
print("obs.t.jd: ",obs.t.jd," obs.tbary.jd: ",obs.tbary.jd)

obs.barytcorr in seconds:  77.84495121886842 s
obs.baryvcorr in km/s: -15.841610801902883 km / s
obs.t.jd:  2458365.602629282  obs.tbary.jd:  2458365.6035302654


**Note carefully** that ```tbary``` is **not in TDB**, or barycentric dynamical time.  It is most likely in the UTC time system (civil clocks follow this) but with a light-travel time correction added.  TDB is an entirely different kettle of fish, offset these days by 69 seconds from UTC and not necessarily corrected for light-travel time.  Aside from the large constant offset, it's different from UTC in that it is a uniform time scale, whereas UTC has leap seconds added from 
tme to time.  TDB does *not* include a correction for light-travel time in and of itself.  As I understand it, it represents the rate of a clock fixed at the earth's mean gravitational potential with respect to the sun.  Time on the real earth has gets ahead and behind this because of the gravitational time dilation changes through the year due to the earth's eccentric orbit around the sun.  This term is of order 1.6 milliseconds, and as far as I know is only practically significant in millisecond pulsar timing.  

In any case, the light travel time correction ```tbary``` should represent accurately the extra time needed for a plane wave of light to make its way to the solar system barycenter, compared to when you observe it.  ```astropy``` provides ways of converting your time to TDB if needed, and the time-of-flight correction from earth-based time to arrival time at the solar system barycenter should be nearly identical in the UTC or TDB systems.

### Summary output methods

Although the main motivation of this work is as a basis for a python skycalc, there are two methods to generate human-readable output that have been used mostly for testing.  They are ```printnow``` and ```printnight```, as follows:

In [55]:
obs.printnow()

 
Site : Anglo-Australian Obs;  E longit = 149 03 57.9322, lat = -31 16 37.3359
 
     J2000:  01 09 18.00  +28 33 52.0       (in Psc)
J2018.702 :  01 10 19.59  +28 39 49.5
 
UT date and time    : Fri 2018-09-14 13:14:15    JD 2458376.0515625 
local date and time : Fri 2018-09-14 23:14:15
 
Local mean sidereal time: 22 44 23 
 
Hour angle: -02 25 56  AltAz:  20.7,   33.9  Parallactic: -147.1 [32.9]
Airmass:  2.802
 
Moon: 2.4 days before first quarter   Alt,Az -8.7, 248.3
The moon is down. 
 
The sun is down; there is no twilight.
Sun RA and dec:  11 29 25.7  +03 17 59 (J2018.702);  AltAz -60.1, 203.0
 
Barycentric corrns: add  385.52 sec and  16.14 km/s to observed.
Barycentric JD (UTC system):  2458376.05602.


As you can see, this generates and prints out the instantaneous circumstances.

Another method, ```printnight```, generates and tabulates the events of the night:

In [56]:
obs.printnight()

Night events; times listed are local.

         Sunset:  Fri 2018-09-14 17:55
  Twilight Ends:  Fri 2018-09-14 19:16
Center of Night:  Fri 2018-09-14 23:59
Twilight Begins:  Sat 2018-09-15 04:43
        Sunrise:  Sat 2018-09-15 06:03
 
        Moonset:  Fri 2018-09-14 22:33
       Moonrise:  Sat 2018-09-15 09:43


Note that **the order in which you call the methods is important**; after setting the input parameters, you first ```computesky``` to compute the sidereal time and some other parameters, then ```computesunmoon```, then ```setnightevents``` and/or ```computeplanets```.  This makes sense; you don't need to compute where the moon is to find the airmass, for example. 

In [6]:
obs.computesky()
obs.computesunmoon()
obs.setnightevents()
print('Sunset (UT):', obs.tsunset, '   Sunrise (UT): ',obs.tsunrise)

Sunset (UT): 2018-09-04 07:49:16.205158    Sunrise (UT):  2018-09-04 20:15:44.691036
